Sentiments analysis


In [ ]:
import mwclient #mlibreria para consultar en wikipedia
import time
import tensorflow

site = mwclient.Site("en.wikipedia.org") # inicia la clase ce mwcliente en wiki 

page = site.pages["Bitcoin"] # va a una pagian de wiki 

In [ ]:
revs = list(page.revisions()) # da una lista de las comentarios de btc 

In [ ]:
revs = sorted(revs, key = lambda rev: rev["timestamp"])  # organiza la lista del primer al ultimo comentario

In [ ]:
revs[0]

finding trge sentiments page 

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text: str)-> list: #el tipo de 'sentimiento' del texto
    sent = sentiment_pipeline([text[:250]])[0] #genera una lista de diccionarios
    score = sent["score"] #puntaje del "sentimiento 1 para posi -1 neg"
    if sent ["label"] =="NEGATIVE": 
        score *=-1
    return score

Average of the sentiment per day 

In [ ]:
edits = {}

for rev in revs: 
    date = time.strftime("%Y-%m-%d", rev["timestamp"])

    if date not in edits:
        edits[date] = dict(sentiments = list(), edit_count = 0)
    
    edits[date]['edit_count'] +=1

    coment = rev["comment"]
    edits[date]['sentiments'].append(find_sentiment(coment))

In [ ]:
print(len(edits))

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"])>0:
        edits[key]["sentiment"]= mean(edits[key]["sentiments"])
        new_edits = []
        #for s in edits[key]["sentiments"]: 
        #    if s < 0: 
        #        new_edits. append(edits[key]["sentiments"])
        #edits[key]["neg_sentiment"]= len(new_edits)/ len/ len(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len( [s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else: 
        edits[key]["sentiment"] = 0 
        edits[key]["sentiments"] = 0 
    del edits[key]["sentiments"] # 


In [ ]:
#Pandas data frame 

import pandas as pd 
edits_df = pd.DataFrame.from_dict(edits, orient= "index")

In [ ]:
edits_df.index = pd.to_datetime(edits_df.index) # convierte la primera columna (str) en fechas (datatime)

In [ ]:
#filling the gaps in comentaries
from datetime import datetime 

dates = pd.date_range(start = "2009-03-08", end = datetime.today())

In [ ]:
edits_df = edits_df.reindex(dates, fill_value=0) # for dates when we have not comments 

In [ ]:
rolling_edits = edits_df.rolling(30).mean() # encuentra el promedio de cada 30 datos
rolling_edits = rolling_edits.drop()

In [ ]:
rolling_edits.to_csv ("wikipedia_edits.csv")